### Install packages

In [2]:
!pip install -qU datasets transformers pinecone-client langchain openai
!pip install -qU langchain-pinecone pinecone-notebooks
!pip install -qU langchain-openai
!pip install -qU langchain_community
!pip install -qU tqdm

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pinecone 5.4.2 requires pinecone-plugin-inference<4.0.0,>=2.0.0, but you have pinecone-plugin-inference 1.1.0 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pinecone-client 5.0.1 requires pinecone-plugin-inference<2.0.0,>=1.0.3, but you have pinecone-plugin-inference 3.1.0 which is incompatible.


### Load dataset

In [3]:
from datasets import load_dataset

ds = load_dataset("qiaojin/PubMedQA", "pqa_artificial")

print(ds)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/5.19k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/233M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/211269 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['pubid', 'question', 'context', 'long_answer', 'final_decision'],
        num_rows: 211269
    })
})


In [7]:
print(ds['train'])

Dataset({
    features: ['pubid', 'question', 'context', 'long_answer', 'final_decision'],
    num_rows: 211269
})


In [8]:
print(ds['train'][0])

{'pubid': 25429730, 'question': 'Are group 2 innate lymphoid cells ( ILC2s ) increased in chronic rhinosinusitis with nasal polyps or eosinophilia?', 'context': {'contexts': ['Chronic rhinosinusitis (CRS) is a heterogeneous disease with an uncertain pathogenesis. Group 2 innate lymphoid cells (ILC2s) represent a recently discovered cell population which has been implicated in driving Th2 inflammation in CRS; however, their relationship with clinical disease characteristics has yet to be investigated.', 'The aim of this study was to identify ILC2s in sinus mucosa in patients with CRS and controls and compare ILC2s across characteristics of disease.', 'A cross-sectional study of patients with CRS undergoing endoscopic sinus surgery was conducted. Sinus mucosal biopsies were obtained during surgery and control tissue from patients undergoing pituitary tumour resection through transphenoidal approach. ILC2s were identified as CD45(+) Lin(-) CD127(+) CD4(-) CD8(-) CRTH2(CD294)(+) CD161(+) c

In [24]:
print(ds['train'][0]['context'].keys())

dict_keys(['contexts', 'labels', 'meshes'])


In [9]:
train_data = ds['train']

In [30]:
def preprocess(item):
    context = " ".join(item["context"]["contexts"])
    return {
        "id": item["pubid"],
        "question": item["question"],
        "context": context,
        "labels": item["context"]["labels"],
        "meshes": item["context"]["meshes"],
        "answer": item["long_answer"]  # Use long_answer for training or retrieval
    }

# Apply preprocessing
processed_data = train_data.map(preprocess)

processed_data = processed_data.remove_columns(["pubid", "long_answer"])

processed_data

Dataset({
    features: ['question', 'context', 'final_decision', 'id', 'labels', 'meshes', 'answer'],
    num_rows: 211269
})

In [15]:
processed_data[1000]['context'].keys()

dict_keys(['contexts', 'labels', 'meshes'])

### Create and connect pinecone index

In [16]:
from google.colab import userdata

In [17]:
import time
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=userdata.get('PINECONE_API_KEY'))

index_name = "llm-assignment"
if index_name not in pc.list_indexes().names():
        pc.create_index(
            name=index_name,
            dimension=1536,
            metric='cosine',
            spec=ServerlessSpec(
                cloud='aws',
                region='us-east-1'
            )
        )
        while not pc.describe_index(index_name).status["ready"]:
          time.sleep(1)

index = pc.Index(index_name)

### Generate embeddings

In [18]:
from openai import OpenAI
# OpenAI.api_key = userdata.get('OPENAI_API_KEY')
client = OpenAI(api_key = userdata.get('OPENAI_API_KEY'))

In [ ]:
# openai.api_key = userdata.get('OPENAI_API_KEY')

In [19]:
# Generate an ID
currentId = 0;
def nextId():
  global currentId
  currentId += 1
  return f"{currentId}"

In [ ]:
# def generate_embedding(text):
#     response = client.embeddings.create(
#         input=text,
#         model="text-embedding-3-small",
#         encoding_format="float"
#     )
#     return response.data[0].embedding

In [ ]:
# for item in processed_data:
#     for context in item['context']['contexts']:
#         vector = generate_embedding(context)
#         metadata = {"question": item["question"], "answer": item["answer"]}
#         index.upsert([(nextId(), vector, metadata)])

### Faster way for embedding

In [20]:
from tqdm import tqdm

In [35]:
def generate_embeddings_batch(texts):
  try:
    response = client.embeddings.create(
      input=texts,
      model="text-embedding-3-small",
      encoding_format="float"
    )
    return [item.embedding for item in response.data]
  except Exception as e:
    print(f"Error generating embeddings: {e}")
    return []

In [36]:
BATCH_SIZE = 128

In [37]:
len(processed_data)

211269

In [43]:
# Calculate the maximum length of the `context` field
max_context_length = max(len(item["context"]) for item in processed_data)

print("Maximum length of the context field:", max_context_length)

Maximum length of the context field: 5658


In [47]:
# Calculate the maximum length of the `answer` field
max_answer_length = max(len(item["answer"]) for item in processed_data)

print("Maximum length of the answer field:", max_answer_length)

Maximum length of the answer field: 3340


In [48]:
# Calculate the maximum length of the `question` field
max_question_length = max(len(item["question"]) for item in processed_data)

print("Maximum length of the question field:", max_question_length)

Maximum length of the question field: 664


In [42]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Initialize the text splitter with chunk size and overlap
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

# Function to preprocess each context and generate chunks
def preprocess_and_chunk(context_data):
    # question, answer, context, final_decision, labels, meshes = context_data
    chunks = splitter.split_text(context_data['context'])
    return [{
        "question": context_data['question'],
        "answer": context_data['answer'],
        "chunk": chunk,
        "final_decision": context_data['final_decision'],
        "labels": context_data['labels'],
        "meshes": context_data['meshes']
    } for chunk in chunks]

# Apply chunking to all contexts
all_chunked_data = []
for context_data in processed_data:
    all_chunked_data.extend(preprocess_and_chunk(context_data))

# Display a sample of the chunked data
print(all_chunked_data[:3])

[{'question': 'Are group 2 innate lymphoid cells ( ILC2s ) increased in chronic rhinosinusitis with nasal polyps or eosinophilia?', 'answer': 'As ILC2s are elevated in patients with CRSwNP, they may drive nasal polyp formation in CRS. ILC2s are also linked with high tissue and blood eosinophilia and have a potential role in the activation and survival of eosinophils during the Th2 immune response. The association of innate lymphoid cells in CRS provides insights into its pathogenesis.', 'chunk': 'Chronic rhinosinusitis (CRS) is a heterogeneous disease with an uncertain pathogenesis. Group 2 innate lymphoid cells (ILC2s) represent a recently discovered cell population which has been implicated in driving Th2 inflammation in CRS; however, their relationship with clinical disease characteristics has yet to be investigated. The aim of this study was to identify ILC2s in sinus mucosa in patients with CRS and controls and compare ILC2s across characteristics of disease. A cross-sectional stu

In [40]:
len(all_chunked_data)

726512

In [54]:
def get_metadata(item):
  return {
        "question": item["question"],
        "answer": item["answer"],
        "chunk": item["chunk"],
        "final_decision": item["final_decision"],
        "labels": item["labels"],
        "meshes": item["meshes"]
    }

# Since I stoped it, i need to retest it from the last row
# if the index is empty, then it will begin from the first
current_vector_count = index.describe_index_stats()
total_vector_count = current_vector_count.total_vector_count
print(total_vector_count)

# stop_index = len(all_chunked_data)
stop_index = 128 * 10
batch_number = round((stop_index // BATCH_SIZE) + 0.4)
for i in range(0, stop_index, BATCH_SIZE):
    j = i + total_vector_count + 100000
    print(f"Processing batch {i // BATCH_SIZE + 1}/{batch_number}")

    batch = all_chunked_data[j:j + BATCH_SIZE]
    print(f"Batch size: {len(batch)}")

    if len(batch) == 0:
        print("Empty batch, skipping...")
        continue

    chunks = [item['chunk'] for item in batch]
    metadata = (get_metadata(item) for item in batch)

    vectors = generate_embeddings_batch(chunks)
    print(f"Generated {len(vectors)} vectors for {len(chunks)} chunks")

    if len(vectors) != len(chunks):
        print("Mismatch in chunks and vectors, skipping batch...")
        continue

    entries = [
        (nextId(), vector, m)
        for (m, chunk, vector) in zip(metadata, chunks, vectors)
    ]

    print(f"Upserting {len(entries)} entries to Pinecone...")
    response = index.upsert(entries)
    print(f"Upsert response: {response}\n")

14080
Processing batch 1/10
Batch size: 128
Generated 128 vectors for 128 chunks
Upserting 128 entries to Pinecone...
Upsert response: {'upserted_count': 128}

Processing batch 2/10
Batch size: 128
Generated 128 vectors for 128 chunks
Upserting 128 entries to Pinecone...
Upsert response: {'upserted_count': 128}

Processing batch 3/10
Batch size: 128
Generated 128 vectors for 128 chunks
Upserting 128 entries to Pinecone...
Upsert response: {'upserted_count': 128}

Processing batch 4/10
Batch size: 128
Generated 128 vectors for 128 chunks
Upserting 128 entries to Pinecone...
Upsert response: {'upserted_count': 128}

Processing batch 5/10
Batch size: 128
Generated 128 vectors for 128 chunks
Upserting 128 entries to Pinecone...
Upsert response: {'upserted_count': 128}

Processing batch 6/10
Batch size: 128
Generated 128 vectors for 128 chunks
Upserting 128 entries to Pinecone...
Upsert response: {'upserted_count': 128}

Processing batch 7/10
Batch size: 128
Generated 128 vectors for 128 ch

In [57]:
index.describe_index_stats()['total_vector_count']

15360

I will skip generating embeddings here for testing purposes, as the previous total_vector_count of 655,055 rows will suffice.

In [56]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    api_key = userdata.get('OPENAI_API_KEY'),
    model="text-embedding-3-small"
)

In [58]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [59]:
dataset_name = "PubMedQA"
assistant_name = "Medical Assistant"
response_style = "clear, concise, and accurate"

system_message = f"""
You are a knowledgeable assistant, {assistant_name}, providing answers to medical questions based on the {dataset_name} dataset.
When answering, ensure to use relevant information from medical research and literature, and base your responses on the provided context or previously shared answers.
If you're unsure about the answer, reply with, "Sorry, I don't know the answer."
Always aim to provide {response_style} information. Avoid making up medical details or providing unsupported claims.
"""

In [60]:
def generateCompletion(text, query):
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": f"Context: {text} Question: {query}"}
        ]
    )
    return response.choices[0].message.content

In [61]:
def queryData(queryVector):
  query_response = index.query(
    vector=queryVector,
    top_k=3,
    include_values=False,
    include_metadata=True
  )
  matches = query_response.matches
  combined_matches = "\n".join([f"{match.metadata['question']} {match.metadata['answer']}" for match in matches])
  # combined_matches = [f"{match.metadata['question']} {match.metadata['answer']}" for match in matches]
  return combined_matches

In [62]:
query = "What is a CT scan?"
query_embedding = generate_embeddings_batch([query])[0]
results = queryData(query_embedding)
print(results)

Is video-Assisted Thoracoscopy Superior to Standard Computer Tomography of the Thorax for Selection of Patients With Spontaneous Pneumothorax for Bullectomy? CT detects ELCs >/=2 cm in 54% of the patients in whom VAT shows these abnormalities. If interventional therapy is deemed appropriate for first time or recurrent SP, VAT should be used since it is superior to CT in demonstrating the presence, number, and size of ELCs.
Do real-time computed tomography image update for endoscopic skull base surgery? The xCAT intra-operative mobile volume computed tomography scanner is a technological advancement that can assist the endoscopic sinus surgeon when performing complex rhinological and skull base procedures.
Is volume of new-onset thrombus associated with the development of postimplantation syndrome after endovascular aneurysm repair? The volume of new-onset thrombus is associated with the development of PIS after EVAR, whereas chronic mural thrombus appears to be an inert material. IMA p

In [63]:
comp = generateCompletion(results, query)

In [64]:
print(comp)

A CT scan, also known as Computed Tomography scan, is a type of imaging test that uses a series of x-ray images taken from different angles around your body and uses computer processing to create cross-sectional images (slices) of the bones, blood vessels, and soft tissues inside your body. These images provide more detailed information than regular x-ray exams, helping doctors to diagnose and treat medical conditions.
